## Introduction

This notebook demonstrates how to visualize and analyze JWST data cubes, including how to:

* Launch [CubeViz](https://jdaviz.readthedocs.io/en/latest/cubeviz/index.html)
* [Load data](https://jdaviz.readthedocs.io/en/latest/cubeviz/import_data.html) from a notebook cell
* Adjust [display parameters](https://jdaviz.readthedocs.io/en/latest/cubeviz/displaycubes.html#display-settings)
* Subtract continuum from a datacube
* Select [spectral regions](https://jdaviz.readthedocs.io/en/latest/specviz/displaying.html#spectral-regions) for further analysis 
* Use the [Moment Maps](https://jdaviz.readthedocs.io/en/latest/cubeviz/plugins.html#moment-maps) plugin to map a spectral feature
* Use the Moment Maps plugin to extract a velocity map
* [Model fitting](https://jdaviz.readthedocs.io/en/latest/cubeviz/plugins.html#model-fitting) for every spaxel in a cube.
* Use a cube model to create feature maps.

Updated on June 10, 2022.


## Data

We use a simulated 4-point dithered observation of a point source produced using [MIRISim](https://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/mirisim), 
with the spectrum of active galaxy NGC 5728.  It has been processed with the [JWST pipeline](https://jwst-pipeline.readthedocs.io/en/latest/) (Detector1/Spec2/Spec3).  The output is constructed only for a 
single MRS channel/band (2C).

For the NIRSpec IFU data, a point source (quasar) was simulated using the NIRSpec Instrument Performance Simulator (IPS), then run through the JWST Spec2 and Spec3 pipelines. 

## Imports 

* [_jdaviz_](https://jdaviz.readthedocs.io/en/latest/) : Cubeviz data visualization tool
* [_astropy.io_](https://docs.astropy.org/en/stable/io/fits/index.html) for reading and writing FITS cubes and images
* [_astropy.utils_](https://docs.astropy.org/en/stable/utils/index.html) for downloading files from URLs

In [ ]:
from jdaviz import Cubeviz
from astropy.io import fits
from astropy.utils.data import download_file

## Visualize a simulated MIRI MRS data cube

* Load and display the data
* Select a spectral region
* Create moment maps

Execute the next cell to launch Cubeviz, then follow the instructions for each task enumerated in the cell directly below the Cubeviz app.

In [ ]:
cubeviz = Cubeviz()
cubeviz.app

### UI Instructions:
#### Task 1:  Load the cube and view it
* Load the MIRI MRS datacube into Cubeviz using the next code cell below
* Open the Display menu 
* In the Layer tab, change the stretch to Logarithmic, 95 percentile to see the target PSF wings.
* Scrub through the cube using the Slice slider.

#### Task 2: Select a spectral region for further analysis
* Select 'No selection (create new)' in the Subsets menu in the Cubeviz top banner.
* Select a spectral subset region in spectrum viewer, using the region selection tool under the Tools icon, centered on a spectral feature of interest (e.g., the 10.5 micron PAH feature)

#### Task 3: Extract an image of a spectral feature using the Moment Maps plugin
* Find the Moment Maps data analysis Plugin by clicking the 'Lego' icon at upper right.
* Select a dataset with the Data dropdown (e.g. SCI extension of our cube).
* Select a spectral region in the Spectral Region dropdown (e.g. Subset 1).
* Enter the Moment (0 for flux, 1 for centroid, 2 for dispersion).
* Press 'Calculate' 
* Use the 2nd cube/image viewer to display the moment 0 image. Click the "Data" button, deselect the 'ERR' extension checkbox, and select the 'Moment' dataset that was generated by the previous step. 
* Adjust the Stretch in the viewer and change the colormap to inferno

#### Task 4: Model the continuum + broad dust features (PAHs) 
* Open the Model Fitting Plugin and resize the tray to make the model parameters readable
* Select the dataset (SCI extension of our cube)
* Create 3 model components: Linear Model L, Gaussian Model G1, and Gaussian Model G2, with the following starting parameters: 
L: Leave starting parameter guess as is    
G1: ampl = 3.5E5 MJy/sr, stddev = 5e-8 m, mean = 1.05E-5 m = fixed  (10.5 um)
G2: ampl = 1.5E6 MJy/sr, stddev = 1e-7 m = fixed, mean = 1.126E-5 m = fixed (11.25 um)
* Enter the model to fit in the Model Equation Editor: L + G1 + G2
* Enter "Model" as the Model Label
* Press Fit
* Press Apply to Cube
* Deselect all data in cube viewer 2, then select Model [Cube] to display the modeled continuum + PAH cube
* Scrub through the Model cube

#### Task 5: Subtract the continuum + PAH model cube from the data cube 
* Run the continuum subtraction cell below, which reads in the science data, gets the model cube from the middle viewer, subtracts the continuum, then loads the continuum-subtracted cube into Cubeviz
* The continuum-subtracted spectrum appears in the spectrum viewer on loading.

#### Task 6: Extract a velocity map from the continuum-subtracted data using the Moment Maps plugin
* Open the Moment Maps Plugin
* Select the continuum-subtracted dataset (Unknown HDU Object [Sci])
* Follow the procedure for Task 3; this time for Moment 1
* Use the 3rd cube/image viewer to display the Moment 1 image



## Load Science Datacube into Cubeviz

In [ ]:
#filename = "l3_ch2-long_s3d.fits"
filename = "https://stsci.box.com/shared/static/ezl49sugxn379wbo4tveuromnde8e6lv.fits"
miricube_file = download_file(filename)
cubeviz.app.load_data(miricube_file)

## Continuum Subtraction

In [ ]:
# Open and inspect the science data with astropy.fits.open
with fits.open(miricube_file, memmap=False) as miricube:
    miricube.info()
    sci = miricube['SCI'].data
    #Retrieve the continuum + PAH model cube from middle cube viewer in Cubeviz
    modeled_cube = cubeviz.app.get_data_from_viewer("uncert-viewer", "Model [Cube] 1")
    #Inspect the data dictionary and retrieve the continuum model flux cube
    print(modeled_cube)
    model_flux = modeled_cube["flux"]
    #Subtract the continuum model from the science data
    sci_contsub = sci-model_flux
    #Load the continuum subtracted data back into Cubeviz
    miricube['SCI'].data = sci_contsub
    cubeviz.app.load_data(miricube)


## Inspect the continuum model parameters

In [ ]:
#c_model = cubeviz.get_models(model_label="Model1")
#print(c_model)
#print(c_model["slope_0"])
cubeviz.get_models()

## NIRSpec IFU

In [ ]:
cubeviz2 = Cubeviz()
cubeviz2.app

In [ ]:
#Download the cube data from Box
filename = "https://stsci.box.com/shared/static/ff0bj31acot1272x5qq2clbmto3s6v4f.fits"
ifucube_file = download_file(filename, cache=True)
cubeviz2.app.load_data(ifucube_file)

## MANGA IFU

In [ ]:
cubeviz3 = Cubeviz()
cubeviz3.app

In [ ]:
fn = download_file('https://stsci.box.com/shared/static/28a88k1qfipo4yxc4p4d40v4axtlal8y.fits', cache=True)
cubeviz3.load_data(fn)